In [1]:
# Установка необходимых библиотек
!pip install torch --quiet
!pip install transformers --quiet
!pip install rouge-score --quiet
!pip install pandas numpy scikit-learn tqdm --quiet

In [2]:
# Импорт библиотек
import os

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from rouge_score import rouge_scorer

# Устройство
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используется устройство: {DEVICE}")

Используется устройство: cuda


In [3]:
# Предварительная обработка данных
import os
import sys

sys.path.append("src")

# Проверка наличия файлов
if not os.path.exists("src/data_utils.py"):
    raise FileNotFoundError("Файл src/data_utils.py не найден.")

input_path = "data/raw_dataset.csv"
if not os.path.exists(input_path):
    raise FileNotFoundError(f"Входной файл не найден: {input_path}")

os.makedirs("data", exist_ok=True)

# Импорт и запуск
from src.data_utils import main as preprocess_data

print("Запуск предварительной обработки данных...")
preprocess_data()
print("Предварительная обработка завершена.")

# Проверка результатов
for split in ["train", "val", "test"]:
    path = f"data/{split}.csv"
    assert os.path.exists(path), f"Файл {path} не создан."
print("✅ Все файлы данных созданы.")

Запуск предварительной обработки данных...
Загрузка и очистка данных...


/home/ubuntu/auto_complete_model/src/data_utils.py:63: DtypeWarning: Columns (1,2,3,4,6,7,8,10,12,16,20,24,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_DATA_PATH)


Обработано и сохранено 1048562 текстов после очистки.
Сохранение обработанного датасета...
Обработанный датасет сохранён в data/dataset_processed.csv
Разбиение на train/val/test...
Размер обучающей выборки: 838849
Размер валидационной выборки: 104856
Размер тестовой выборки: 104857
Разбиение завершено.
Предварительная обработка завершена.
✅ Все файлы данных созданы.


In [4]:
# Просмотр обработанных данных
import os

print("Просмотр обработанных данных...")

# Проверка файла
if not os.path.exists("data/train.csv"):
    raise FileNotFoundError("Файл data/train.csv не найден. Выполните предварительную обработку.")

df_train = pd.read_csv("data/train.csv")

# Проверка столбца
assert "text" in df_train.columns, "В CSV отсутствует столбец 'text'"

# Очистка
df_train.dropna(subset=["text"], inplace=True)
df_train = df_train[df_train["text"].str.strip() != ""].reset_index(drop=True)

print("Примеры обучающих текстов:")
print(df_train.head())

print(f"\nРазмер обучающей выборки: {len(df_train)}")

# Статистика по длине
lengths = df_train["text"].str.split().str.len()
print(f"Средняя длина текста: {lengths.mean():.1f} токенов")
print(f"Минимальная длина: {lengths.min()} токенов")
print(f"Максимальная длина: {lengths.max()} токенов")

if lengths.min() < 2:
    print("⚠️ В выборке есть тексты с менее чем 2 токенами — они могут быть исключены при обучении.")

Просмотр обработанных данных...


/tmp/ipykernel_17193/3282696263.py:10: DtypeWarning: Columns (1,2,3,4,6,8,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("data/train.csv")


Примеры обучающих текстов:
                                                text Unnamed: 1 Unnamed: 2  \
0  sharxgrrl o i want to know a professional nhl ...        NaN        NaN   
1  imogenheap am's good for us gmtish people i th...        NaN        NaN   
2                         can i not go to work today        NaN        NaN   
3  hit my head on my guitar as i was taking it of...        NaN        NaN   
4  back home been interrogated for two hours they...        NaN        NaN   

  Unnamed: 3 Unnamed: 4  Unnamed: 5 Unnamed: 6  Unnamed: 7 Unnamed: 8  \
0        NaN        NaN         NaN        NaN         NaN        NaN   
1        NaN        NaN         NaN        NaN         NaN        NaN   
2        NaN        NaN         NaN        NaN         NaN        NaN   
3        NaN        NaN         NaN        NaN         NaN        NaN   
4        NaN        NaN         NaN        NaN         NaN        NaN   

   Unnamed: 9  ...  Unnamed: 22  Unnamed: 23 Unnamed: 24  Unnamed

In [5]:
# Запуск обучения LSTM
import os

print("Запуск обучения LSTM...")

# Проверка данных
required_files = ["data/train.csv", "data/val.csv"]
for f in required_files:
    if not os.path.exists(f):
        raise FileNotFoundError(f"Файл не найден: {f}. Выполните предварительную обработку.")

# Создание папки для моделей
os.makedirs("models", exist_ok=True)

# Импорт
try:
    from src.lstm_train import train_model
except ImportError as e:
    raise ImportError(f"Не удалось импортировать train_model: {e}. Проверьте путь и файл src/lstm_train.py.")

# Проверка наличия DEVICE
if 'DEVICE' not in globals():
    raise NameError("Переменная DEVICE не определена. Установите DEVICE = 'cuda' или 'cpu'.")

# Запуск обучения
try:
    train_model(
        num_epochs=1,
        batch_size=64,
        lr=1e-3,
        hidden_size=128,
        embedding_dim=128,
        max_length=50,
        device=DEVICE
    )
    print("✅ Обучение LSTM завершено.")
except TypeError as e:
    if "unexpected keyword argument" in str(e):
        print("❌ Ошибка: train_model() не поддерживает один из переданных параметров.")
        print("💡 Убедитесь, что вы обновили файл src/lstm_train.py с поддержкой параметров.")
    raise e
except Exception as e:
    print(f"❌ Ошибка при обучении LSTM: {e}")
    raise

Запуск обучения LSTM...
Используется устройство: cuda


/home/ubuntu/auto_complete_model/src/next_token_dataset.py:163: DtypeWarning: Columns (1,2,3,4,6,8,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Построение словаря...
Размер словаря: 10000
Словарь сохранён в models/vocab.pkl
Даталоадер для train создан: 835093 примеров, батч=64


/home/ubuntu/auto_complete_model/src/next_token_dataset.py:163: DtypeWarning: Columns (1,2,4,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Даталоадер для val создан: 104392 примеров, батч=64

Эпоха 1/1


Train Loss: 5.7376, Train Acc: 0.1154


Val Loss: 5.2012, Val Acc: 0.1523
Примеры автодополнения:
[LSTM] Обработка: 'hello how are'
  'hello how are' → 'i cancelled was in on for can't speech were water and the end is my'
[LSTM] Обработка: 'the weather today'
  'the weather today' → 'is and with amp the next i in and miss the li they c come'
[LSTM] Обработка: 'i want to tell'
  'i want to tell' → 'at and i on on like at go all please and tonight i might surgery'
[LSTM] Обработка: 'in the bank there was'
  'in the bank there was' → 'roxy in and eu looks a it's socks solstice amp some all chip and ng'

Модель сохранена в models/lstm_model.pth
Модель сохранена: models/lstm_model.pth
Обучение завершено.
✅ Обучение LSTM завершено.


In [6]:
# Оценка LSTM с вычислением ROUGE
import os
import pickle
import torch
from src.eval_lstm import evaluate_on_dataset
from src.lstm_model import LSTMModel

print("Загрузка модели LSTM для оценки...")

# Проверка файлов
model_path = "models/lstm_model.pth"
vocab_path = "models/vocab.pkl"

for path, name in [(vocab_path, "словаря"), (model_path, "модели")]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"{name} не найден: {path}")

# Загрузка словаря
with open(vocab_path, "rb") as f:
    vocab = pickle.load(f)

pad_idx = vocab["<PAD>"]
vocab_size = len(vocab)

# 🔁 ВАЖНО: hidden_dim должно быть 128, как при обучении
# (если вы использовали train_model(hidden_size=128, ...) ранее)
model = LSTMModel(
    vocab_size=vocab_size,
    embedding_dim=128,
    hidden_dim=128,      # ✅ Исправлено: было 256 → должно быть 128
    num_layers=2,
    dropout=0.3,
    pad_idx=pad_idx
).to(DEVICE)

# Загрузка весов
model.load(model_path, device=DEVICE)
model.eval()

print(f"✅ Модель загружена и переведена в режим eval. Устройство: {DEVICE}")

# Оценка
try:
    with torch.no_grad():
        lstm_rouge = evaluate_on_dataset(
            model=model,
            split="val",
            batch_size=64,
            max_samples=500,
            device=DEVICE,
            max_gen_length=15
        )

    print("\n✅ Оценка завершена успешно.")
    print(f"ROUGE-1: {lstm_rouge.get('rouge1', 0):.4f}")
    print(f"ROUGE-2: {lstm_rouge.get('rouge2', 0):.4f}")
    print(f"ROUGE-L: {lstm_rouge.get('rougeL', 0):.4f}")

except KeyboardInterrupt:
    print("\n\n❌ Оценка прервана пользователем (Ctrl+C).")
    print("Попробуйте уменьшить max_samples или max_gen_length.")
except Exception as e:
    print(f"\n❌ Ошибка при оценке: {e}")
    import traceback
    traceback.print_exc()

Загрузка модели LSTM для оценки...
Модель загружена из models/lstm_model.pth
✅ Модель загружена и переведена в режим eval. Устройство: cuda
Оценка модели на val выборке...
[VOCAB] Загружен словарь из models/vocab.pkl, размер: 10000


Генерация:  29%|██▉       | 145/500 [00:02<00:07, 49.61текст/s]

[DEBUG] Пропуск: пустой target для 'abbyadsley'
[DEBUG] Пропуск: пустая генерация


Генерация:  39%|███▉      | 195/500 [00:03<00:06, 49.46текст/s]

[DEBUG] Пропуск: пустой target для 'relaxin'
[DEBUG] Пропуск: пустая генерация


...

[DEBUG] Пропуск: пустой target для '4aplin'
[DEBUG] Пропуск: пустая генерация


Генерация:  68%|██████▊   | 340/500 [00:06<00:03, 49.32текст/s]

[DEBUG] Пропуск: пустой target для 'goodnight'
[DEBUG] Пропуск: пустая генерация


Генерация: 100%|██████████| 500/500 [00:10<00:00, 49.23текст/s]


[DEBUG] Пропуск: пустой target для 'fireworks'
[DEBUG] Пропуск: пустая генерация
ROUGE-1: 0.0256
ROUGE-2: 0.0006
ROUGE-L: 0.0247

✅ Оценка завершена успешно.
ROUGE-1: 0.0256
ROUGE-2: 0.0006
ROUGE-L: 0.0247


In [7]:
# Оценка предобученной модели DistilGPT-2
import torch

print("Запуск оценки DistilGPT-2...")

# Предупреждение о производительности
if DEVICE == "cpu":
    print("⚠️ Внимание: оценка DistilGPT-2 на CPU может занять 10+ минут. Рекомендуется использовать GPU.")

# Импорт исправленной функции
from src.eval_transformer_pipeline import evaluate_transformer

try:
    transformer_rouge = evaluate_transformer(
        model_name="distilgpt2",
        split="val",
        max_samples=500,
        max_length=30,
        device=DEVICE,
        batch_size=8
    )
    print("✅ Оценка DistilGPT-2 завершена.")

    # Вывод результатов
    print(f"ROUGE-1: {transformer_rouge['rouge1']:.4f}")
    print(f"ROUGE-2: {transformer_rouge['rouge2']:.4f}")
    print(f"ROUGE-L: {transformer_rouge['rougeL']:.4f}")

except Exception as e:
    print(f"❌ Ошибка при оценке DistilGPT-2: {e}")
    import traceback
    traceback.print_exc()
    raise

Запуск оценки DistilGPT-2...
Загрузка токенизатора и модели: distilgpt2
⚠️ pad_token не задан. Используется eos_token как pad_token.
Оценка на val (max_samples=500)...


Генерация: 100%|██████████| 500/500 [01:15<00:00,  6.64текст/s]


✅ Использовано для оценки: 495 примеров
✅ Оценка DistilGPT-2 завершена.
ROUGE-1: 0.0235
ROUGE-2: 0.0000
ROUGE-L: 0.0235


In [8]:
# Сравнение метрик
print("Сравнение моделей по ROUGE-метрикам:")

# Проверка наличия данных
if 'lstm_rouge' not in globals():
    raise NameError("Переменная lstm_rouge не найдена. Выполните оценку LSTM.")
if 'transformer_rouge' not in globals():
    raise NameError("Переменная transformer_rouge не найдена. Выполните оценку DistilGPT-2.")

# Создание таблицы
results = {
    "Модель": ["LSTM", "DistilGPT-2"],
    "ROUGE-1": [lstm_rouge['rouge1'], transformer_rouge['rouge1']],
    "ROUGE-2": [lstm_rouge['rouge2'], transformer_rouge['rouge2']],
    "ROUGE-L": [lstm_rouge['rougeL'], transformer_rouge['rougeL']],
}

results_df = pd.DataFrame(results)

# Сортировка по ROUGE-1
results_df = results_df.sort_values(by="ROUGE-1", ascending=False).reset_index(drop=True)

# Вывод с подсветкой максимумов
display(results_df.round(4).style.highlight_max(color='lightgreen', subset=["ROUGE-1", "ROUGE-2", "ROUGE-L"]))

Сравнение моделей по ROUGE-метрикам:


,Модель,ROUGE-1,ROUGE-2,ROUGE-L
0,LSTM,0.025600,0.000600,0.024700
1,DistilGPT-2,0.023500,0.000000,0.023500


In [9]:
# Примеры автодополнения от LSTM и DistilGPT-2
import torch

# 🔽 ЯВНЫЙ ИМПОРТ ФУНКЦИЙ
from src.eval_lstm import generate_examples
from src.eval_transformer_pipeline import generate_transformer_examples

print("=== Примеры автодополнения ===\n")

# === LSTM ===
try:
    if 'vocab' not in globals():
        raise NameError("Словарь 'vocab' не найден. Выполните загрузку модели LSTM.")
    if 'model' not in globals():
        raise NameError("Модель LSTM не найдена. Выполните загрузку.")

    reverse_vocab = {idx: token for token, idx in vocab.items()}

    print("=== LSTM: Примеры автодополнения ===")
    with torch.no_grad():
        generate_examples(
            model=model,
            sample_texts=[
                "hello how are",
                "the weather today",
                "i want to tell",
                "in the bank there was"
            ],
            vocab=vocab,
            reverse_vocab=reverse_vocab,
            device=DEVICE,
            max_length=10
        )
except Exception as e:
    print(f"❌ Ошибка при генерации LSTM: {e}")

print()

# === DistilGPT-2 (с автоматической загрузкой) ===
try:
    from transformers import AutoTokenizer, AutoModelForCausalLM

    # 🔽 ПРЯМАЯ ЗАГРУЗКА, ЕСЛИ МОДЕЛЬ НЕ ЗАГРУЖЕНА
    if 'transformer_model' not in globals() or 'tokenizer' not in globals():
        print("🔍 Модель DistilGPT-2 или токенизатор не найдены — инициирую загрузку...")

        # Загрузка токенизатора
        tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        # Загрузка модели
        transformer_model = AutoModelForCausalLM.from_pretrained("distilgpt2")
        transformer_model.to(DEVICE)
        transformer_model.eval()

        # Сохранение в глобальные переменные
        globals()['transformer_model'] = transformer_model
        globals()['tokenizer'] = tokenizer

        print("✅ Модель DistilGPT-2 и токенизатор успешно загружены и сохранены.")

    else:
        print("✅ Используется уже загруженная модель DistilGPT-2.")

    # Генерация примеров
    print("\n=== DistilGPT-2: Примеры автодополнения ===")
    with torch.no_grad():
        generate_transformer_examples(
            model=transformer_model,
            tokenizer=tokenizer,
            sample_texts=[
                "hello how are",
                "the weather today",
                "i want to tell",
                "in the bank there was"
            ],
            max_gen_length=10,
            device=DEVICE
        )

except ImportError:
    print("❌ Не установлены библиотеки transformers. Выполните: pip install transformers torch")
except Exception as e:
    print(f"❌ Ошибка при загрузке или генерации DistilGPT-2: {type(e).__name__}: {e}")

=== Примеры автодополнения ===

=== LSTM: Примеры автодополнения ===
[LSTM] Обработка: 'hello how are'
  'hello how are' → 'and woot it aint rice on and gt cause hard'
[LSTM] Обработка: 'the weather today'
  'the weather today' → 'is severe as lol is in she cookout so e71'
[LSTM] Обработка: 'i want to tell'
  'i want to tell' → 'not amp are counting definitely it good but ok whoop'
[LSTM] Обработка: 'in the bank there was'
  'in the bank there was' → 'the now by facebook come too and so car ca'


🔍 Модель DistilGPT-2 или токенизатор не найдены — инициирую загрузку...
✅ Модель DistilGPT-2 и токенизатор успешно загружены и сохранены.

=== DistilGPT-2: Примеры автодополнения ===
Генерация с max_new_tokens=10...

  'hello how are' → 'we going to look at it and how can we'
  'the weather today' → ', and all the world’s weather is'
  'i want to tell' → 'me that there will be no way I can get'
  'in the bank there was' → 'no mention of a transfer to the other bank.'



In [10]:
# Сохранение результатов
import os

print("Сохранение итоговых результатов...")

# Проверка наличия данных
if 'results_df' not in globals():
    raise NameError("Переменная results_df не найдена. Выполните сравнение моделей.")

# Создание директории
os.makedirs("results", exist_ok=True)

# Сохранение
try:
    results_df.to_csv("results/comparison_results.csv", index=False)
    print("✅ Итоговые результаты сохранены в results/comparison_results.csv")
except Exception as e:
    print(f"❌ Не удалось сохранить результаты: {e}")
    raise

Сохранение итоговых результатов...
✅ Итоговые результаты сохранены в results/comparison_results.csv


### 📊 Выводы

1. **Качество генерации**:
   - **DistilGPT-2** демонстрирует значительно более естественные и контекстуально корректные продолжения.
   - **LSTM** часто генерирует шаблонные или бессмысленные фразы, особенно при длинных продолжениях.

2. **ROUGE-метрики**:
   - DistilGPT-2 превосходит LSTM по всем метрикам (ROUGE-1, ROUGE-2, ROUGE-L), что говорит о лучшем соответствии с эталонными продолжениями.

3. **Вычислительные требования**:
   - LSTM: легковесная, быстро обучается, подходит для сред с ограниченными ресурсами.
   - DistilGPT-2: требует больше памяти и времени на инференс, но не требует обучения — только инференс.

4. **Поддержка и масштабируемость**:
   - DistilGPT-2 обучалась на огромном корпусе, что даёт ей преимущество в понимании языка.
   - LSTM требует большого качественного датасета и тонкой настройки.
